In [9]:
from qiskit_ibm_provider import IBMProvider
IBMProvider.save_account('f4ff8b579fea0cd17500e9638f711a685bf3b399f8637a6734e66cb6453d3500a634b4456043973e7d79695c22b5d40282e618f6ba7f106cd0e5d7cccfbf73c0')
provider = IBMProvider(instance = 'ibm-q/open/main')

In [16]:
from qiskit_ibm_provider import least_busy
[b for b in provider.backends() if not b.configuration().simulator]

[<IBMBackend('ibmq_quito')>,
 <IBMBackend('ibmq_jakarta')>,
 <IBMBackend('ibmq_manila')>,
 <IBMBackend('ibm_lagos')>,
 <IBMBackend('ibm_oslo')>,
 <IBMBackend('ibm_perth')>,
 <IBMBackend('ibmq_lima')>,
 <IBMBackend('ibmq_belem')>,
 <IBMBackend('ibm_nairobi')>]

In [23]:
provider.backends(
    filter=lambda b: not b.configuration().simulator
                     and b.configuration().num_qubits > 5
)


[]

In [18]:
backends = [b for b in provider.backends() if not b.configuration().simulator]

In [32]:
min_gate_error = 1
for backend in backends:
    if 'cx' not in backend.configuration().basis_gates:
    	continue
    for qubit_pair in backend.configuration().coupling_map:
        gate_error = backend.properties().gate_error('cx', qubit_pair)
        if gate_error < min_gate_error:
            min_gate_error = gate_error
            best_qubits = qubit_pair
            best_backend = backend
            print(f"Lowest cx error is between qubits {best_qubits} in '{best_backend.name}' (error: {min_gate_error:.5f})")


Lowest cx error is between qubits [0, 1] in 'ibmq_quito' (error: 0.00883)
Lowest cx error is between qubits [0, 1] in 'ibmq_jakarta' (error: 0.00780)
Lowest cx error is between qubits [3, 5] in 'ibmq_jakarta' (error: 0.00520)
Lowest cx error is between qubits [1, 3] in 'ibm_lagos' (error: 0.00518)


In [ ]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

# Load IBMQ account
IBMQ.load_account()

# Choose the backend
backend = IBMQ.get_provider().get_backend('ibm_lagos')

# Define quantum circuit
qc = QuantumCircuit(2)
qc.h(0)  # Apply Hadamard gate to qubit 0

# Transpile the circuit for the chosen backend
qc_transpiled = transpile(qc, backend=backend)

# Assemble the transpiled circuit into a Qobj
qobj = assemble(qc_transpiled)

# Submit the job to the backend and wait for results
job = backend.run(qobj)
job_result = job.result()

# Get the counts from the result
counts = job_result.get_counts()
print(counts)

# Plot the histogram of the measurement outcomes
plot_histogram(counts)
